In [1]:
import yaml
import os
from pathlib import Path
import pandas as pd
import struct
import random
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
import re

# also: cd hammer && pip install -e . 

In [2]:
# experiment setup
PDK = 'sky130'
CLOCK_PERIOD = 10 if PDK == 'sky130' else 2 # ns

# generate custom make str for each test
make_extra = f"pdk={PDK}"
if PDK == 'intech22': make_extra += f" PDK_CONF=experiments/intech22.yml"

# useful paths
energy_char_dpath = Path(os.getcwd()).parent
tests_dpath = energy_char_dpath/f'experiments/tests-{PDK}'
tests_dpath

PosixPath('/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130')

In [3]:
# get pdk clock periods, paths
PDKs = ["sky130"]
test_paths = {}
for pdk in PDKs:
    test_paths[pdk] = energy_char_dpath/f'experiments/tests-{pdk}'

operations = ['regfile']
modules = {op:op for op in operations}
module_to_inst = {'regfile':'regfile'}
clock_name = {'regfile':'clk'} ## name of clock for respective module name
num_inputs, widths, num_registers = 1, [64, 64, 64, 8], [8, 32, 64, 32]
clock_periods = [CLOCK_PERIOD] # ns, clock periods to apply to all designs; eventually try sweeping this


design_minclock_dict = {}
# optional:
# design_minclock_dict={'add4': 4, 'add8': 4, 'add16': 4, 'add32': 4, 'add64': 4, 'mul4': 5, 'mul8': 5, 'mul16': 6, 'mul32': 6, 'mul64': 5}

'''
tests dict
  name format: <design>-<test_name>
      inputs: list, where each item is a tuple of items per line in input.txt
      defines: for verilog
'''

'\ntests dict\n  name format: <design>-<test_name>\n      inputs: list, where each item is a tuple of items per line in input.txt\n      defines: for verilog\n'

In [4]:
def writeYaml(td):
    defines_str = '\n'.join( [ f"  - {key}={val}" for key,val in td['defines'].items() ] )
    clock_period = td["clock_period"]
    delays = [f"""{{name: {i}, clock: {td['clock']}, delay: "1", direction: input}}""" for i in td['input_ports']]
    delays += [f"""{{name: {i}, clock: {td['clock']}, delay: "1", direction: output}}""" for i in td['output_ports']]
    delays = ',\n  '.join(delays)
    cfg = f"""\
vlsi.core.build_system: make
vlsi.inputs.power_spec_type: cpf
vlsi.inputs.power_spec_mode: auto

design.defines: &DEFINES
  - CLOCK_PERIOD={clock_period}
{defines_str}

vlsi.inputs.clocks: [{{name: {td['clock']}, period: "{clock_period}ns", uncertainty: "100ps"}}]

vlsi.inputs.delays: [
  {delays}
]

synthesis.inputs:
  top_module: {td['top_module']}
  input_files: {td['vsrcs']}
  defines: *DEFINES

sim.inputs:
  top_module: {td['top_module']}
  tb_name: {td['tb_name']}
  tb_dut: {td['tb_dut']}
  options: ["-timescale=1ns/10ps", "-sverilog"]
  options_meta: append
  defines: *DEFINES
  defines_meta: append
  level: rtl
  input_files: {td['vsrcs'] + td['vsrcs_tb']}

vlsi.core.power_tool: hammer.power.joules
power.inputs:
  level: rtl
  top_module: {td['top_module']}
  tb_name: {td['tb_name']}
  tb_dut: {td['tb_dut']}
  defines: *DEFINES
  input_files: {td['vsrcs']}
  report_configs:
    - waveform_path: {td['root']}/output.fsdb
      report_stem: {td['root']}/power
      toggle_signal: {td['clock']}
      num_toggles: 1
      levels: all
      output_formats:
      - report
      - plot_profile
      - ppa
"""
    with (td['root']/'config.yml').open('w') as f:
        f.write(cfg)

In [5]:
def formattedOp(op):
    if (op == 'regfile'):
        return 'regfile'


# Reset regs to 0 or rand
def get_initial_vals(n, width, rand=True):
    vals = []
    for i in range(n):
        rand_val = random.randint(0, 1 << width - 1)
        vals.append([0, 0, 1, i, rand_val if rand else 0])
    return vals

def writeTest(n, width, rand=True):
    init_vals = get_initial_vals(n, width, False) # always reset to regs to 0
    ret = []
    for i in range(n):
        rand_val = random.randint(0, 1 << width - 1)
        # r_en, r_addr, w_en, w_addr, w_data 
        ret.append([0, 0, 1, i, rand_val if rand else 1]) # write 1 or rand val
    return init_vals, ret # reset.txt, input.txt vals

def readTest(n, width, rand=True):
    init_vals = get_initial_vals(n, width, rand) # reset to rand or 0
    ret = []
    for i in range(n):
        # r_en, r_addr, w_en, w_addr, w_data 
        ret.append([1, i, 0, 0, 0])
    return init_vals, ret

def resetTest(n, width, rand=True):
    init_vals = get_initial_vals(n, width, rand)
    ret = []
    for i in range(n):
        # r_en, r_addr, w_en, w_addr, w_data 
        ret.append([0, 0, 1, i, 0])
    return init_vals, ret


def createTest(operation, n, width, clock_period, test_type, rand=False):
    tt_to_name = {resetTest: 'reset', readTest: 'read', writeTest: 'write'}
    design = f"{formattedOp(operation)}{n}x{width}-{clock_period}ns"
    test_name = f'{design}-{tt_to_name[test_type]}'
    if (rand):
        test_name = f'{test_name}-rand'


    reset_vals, inputs = test_type(n, width, rand)
    print("Operation, #registers, width:", operation, n, width)
    print("For test type:", test_type)
    print("Reset values:", reset_vals)
    print("Input values:", inputs)
    ## set inputs
    new_test = {'inputs': inputs, 'resets': reset_vals, 'defines': dict(WIDTH=width)}

    new_test['num_registers'] = n
    new_test['width'] = width

    ## verilog info
    module = modules[operation]
    new_test['design'] = design
    new_test['inst'] = f'/{module_to_inst[module]}'
    new_test['clock'] = clock_name[module]
    new_test['vsrcs'] = [f'src/{module}.v']
    new_test['vsrcs_tb'] = [f'src/{module}_tb.v']
    new_test['top_module'] = f'{module}'
    new_test['tb_name'] = f'{module}_tb'
    new_test['tb_dut'] = f'{module}_dut'
    new_test['input_ports'] = ['R_addr', 'R_en', 'R_data', 'W_addr', 'W_en', 'W_data']
    new_test['output_ports'] = ['R_data']
    new_test['read'] = (test_type == readTest)
    new_test['clock_period'] = clock_period

    ## root dir
    root = tests_dpath/test_name
    root.mkdir(exist_ok=True,parents=True)
    new_test['defines']['TESTROOT'] = root
    new_test['defines']['WIDTH'] = width
    new_test['defines']['N'] = n
    new_test['defines']['read'] = new_test['read']
    new_test['root'] = root

    ## design dir
    new_test['obj_dpath'] = energy_char_dpath/f"build-{PDK}-cm/{design}"

    cfg = str(new_test['root']/'config.yml')
    new_test['make'] = f"design={new_test['design']} {make_extra} DESIGN_CONF={cfg}"

    return test_name, new_test

# convert data operands to binary format to dump to input.txt (gets more complicated for floats)
def val2binary(val) -> str:
    if type(val) == str: return val
    elif type(val) == int: return '{0:b}'.format(val)
    else: assert(False), f"Invalid dtype, {type(val)}"

## write inputs to test file
def writeInput(test_info):
    with (test_info['root']/'input.txt').open('w') as f:
        for operands in test_info['inputs']:
            f.write(" ".join([val2binary(operand) for operand in operands]) + '\n')

    with (test_info['root']/'reset.txt').open('w') as f:
        for operands in test_info['resets']:
            f.write(" ".join([val2binary(operand) for operand in operands]) + '\n')
    

In [6]:
tests_dict = {}
for i in range(len(operations)):
    operation = operations[i]
    width = widths[i]
    for n in num_registers:
        design = operation
        design_clock_periods = clock_periods.copy()
        if design in design_minclock_dict:
            design_clock_periods.append(design_minclock_dict[design])
        for clock_period in design_clock_periods:
                    for tt in [resetTest, readTest, writeTest]:
                        for rand_val in [True, False]:
                            print("Working on", operation, n, width, clock_period)
                            test_name, test_info = createTest(operation, n, width, clock_period, tt, rand_val)
                            writeInput(test_info)
                            writeYaml(test_info)
                            tests_dict[test_name] = test_info
                            print(test_info)

Working on regfile 8 64 10
Operation, #registers, width: regfile 8 64
For test type: <function resetTest at 0x7f718e2785e0>
Reset values: [[0, 0, 1, 0, 307198183863669947], [0, 0, 1, 1, 6888527724307962420], [0, 0, 1, 2, 893216917279408630], [0, 0, 1, 3, 7647789146860801221], [0, 0, 1, 4, 3059492980993331690], [0, 0, 1, 5, 5713646859114018481], [0, 0, 1, 6, 7776606061478728720], [0, 0, 1, 7, 7626104092215319755]]
Input values: [[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 2, 0], [0, 0, 1, 3, 0], [0, 0, 1, 4, 0], [0, 0, 1, 5, 0], [0, 0, 1, 6, 0], [0, 0, 1, 7, 0]]
{'inputs': [[0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 0, 1, 2, 0], [0, 0, 1, 3, 0], [0, 0, 1, 4, 0], [0, 0, 1, 5, 0], [0, 0, 1, 6, 0], [0, 0, 1, 7, 0]], 'resets': [[0, 0, 1, 0, 307198183863669947], [0, 0, 1, 1, 6888527724307962420], [0, 0, 1, 2, 893216917279408630], [0, 0, 1, 3, 7647789146860801221], [0, 0, 1, 4, 3059492980993331690], [0, 0, 1, 5, 5713646859114018481], [0, 0, 1, 6, 7776606061478728720], [0, 0, 1, 7, 7626104092215319

In [7]:
# Setup
def runMakeCmd(make_target,td,fp,overwrite=False,verbose=False):
    if overwrite or not fp.exists():
        cmd = f"make {make_target} {td['make']}"
        print(f'Executing commmand: {cmd}')
        subprocess.run(cmd, cwd=energy_char_dpath,
                        shell=True, check=True, capture_output=(not verbose))

# subprocess.run starts in an empty environment, need to ensure it can find hammer-vlsi
python_exec_fpath = Path(sys.executable)
env_dpath = str(python_exec_fpath.parent)
if not os.environ['PATH'].startswith(env_dpath): os.environ['PATH'] = env_dpath + ':' + os.environ['PATH']

def runBuild(td,overwrite=False,verbose=False):
    runMakeCmd("build -B",td,
                td['obj_dpath'],
                overwrite,verbose)

def runSim(td,overwrite=False,verbose=False):
    runMakeCmd("redo-sim-rtl",td,
                td['root']/'output.fsdb',
                overwrite,verbose)

def runPowerSyn(td,overwrite=False,verbose=False):
    runMakeCmd("power-rtl",td,
                td['obj_dpath']/'power-rtl-rundir/pre_report_power',
                overwrite,verbose)

def runPowerReport(td,overwrite=False,verbose=False):
    runMakeCmd("redo-power-rtl args='--only_step report_power'",td,
                td['root']/'power.power.rpt',
                overwrite,verbose)


In [8]:
# build
overwrite = True
build_dpaths = {td['obj_dpath']: t for t,td in tests_dict.items()} # run build once per build dir (not once per test)
for bd,t in build_dpaths.items():
    runBuild(tests_dict[t],overwrite)

Executing commmand: make build -B design=regfile8x64-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x64-10ns-write/config.yml
Executing commmand: make build -B design=regfile32x64-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile32x64-10ns-write/config.yml
Executing commmand: make build -B design=regfile64x64-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile64x64-10ns-write/config.yml


In [9]:
# power-rtl: synthesize design in power tool + save checkpoint
overwrite = True
for bd,t in build_dpaths.items():
    runSim(tests_dict[t],overwrite)
    runPowerSyn(tests_dict[t],overwrite)
    runPowerReport(tests_dict[t],overwrite)

Executing commmand: make redo-sim-rtl design=regfile8x64-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x64-10ns-write/config.yml
Executing commmand: make power-rtl design=regfile8x64-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x64-10ns-write/config.yml
Executing commmand: make redo-power-rtl args='--only_step report_power' design=regfile8x64-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x64-10ns-write/config.yml
Executing commmand: make redo-sim-rtl design=regfile32x64-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile32x64-10ns-write/config.yml
Executing commmand: make power-rtl design=regfile32x64-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb2

In [10]:
# TODO: check ppa.rpt & adjust clock period, print updated values
#       in a dict that user copies to the top? it's a bit messy but we prolly don't want to auto-loop

def parseSlack(fname):
    slacks = []
    with open(fname, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.split()
            if (line[0] == 'register'):
                slacks.append(float(line[6]))
    return min(slacks)

overwrite = True
new_clocks = {}
for bd,t in build_dpaths.items():
    cur_test = tests_dict[t]
    clock_period = cur_test['clock_period']
    slack = parseSlack(cur_test['root']/'power.ppa.rpt')
    if 0 <= slack and slack < 500: slack = 0 # don't change clock period
    new_clock_period = math.floor((clock_period * 1000 - slack) / 1000)
    new_clocks[t.split('-')[0]] = new_clock_period

print(f"design_minclock_dict={new_clocks}")
# copy output to top if desired

design_minclock_dict={'regfile8x64': 4, 'regfile32x64': 4, 'regfile64x64': 4}


In [11]:
# sim-rtl
for t,td in tests_dict.items():
    runSim(td, overwrite=True, verbose=True)
    runPowerReport(td, overwrite=True, verbose=True)

Executing commmand: make redo-sim-rtl design=regfile8x64-10ns pdk=sky130 DESIGN_CONF=/bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x64-10ns-reset-rand/config.yml
/bwrcq/scratch/shreyas_thumathy/miniforge3/bin/hammer-vlsi -e /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/configs-env/bwrc-env.yml -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/configs-pdk/sky130.yml -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/configs-tool/cm.yml -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/experiments/tests-sky130/regfile8x64-10ns-write/config.yml -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/build-sky130-cm/regfile8x64-10ns/sram_generator-output.json  -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/../configs-pdk/sky130.yml  -p /bwrcq/scratch/shreyas_thumathy/feb25/hammer/e2e/hammer-energy-char/../configs-tool/cm.yml  -p /bwrcq/scratch/shreyas_thumathy/feb25/hamm

In [12]:
def parse_hier_power_rpt(td) -> list:
    fpath = td['root']/'power.hier.power.rpt'
    with fpath.open('r') as f: lines = f.readlines()
    for l in lines:
        words = l.split()
        if l.startswith('Power Unit'):
            assert(words[-1] == 'mW'), f"Wrong power unit in report, {l}"
        if td['inst'] == words[-1]:
            #print([float(p) for p in words[2:6]])
            return [float(p) for p in words[2:6]]
    return []

def parse_power_profile(td) -> float:
    fpath = td['root']/'power.profile.png.data'
    with fpath.open('r') as f: lines = f.readlines()
    header = lines[0]
    assert((f"-ykeylabel {td['inst']}:total:total" in header) or (f"-ykeylabel {td['inst'][1:]}:total:total" in header)) # make sure we got the correct power trace
    match = re.search(r'simulation time \((\w+)\)', header)
    unit = match.group(1) if match else None
    if unit == 'ns': scaling = 1
    elif unit == 'ps': scaling = 1e-3
    elif unit == 'fs': scaling = 1e-6
    else: raise ValueError(f"Unit {unit} not supported in file {fpath}")
    time_power = [l.split() for l in lines]
    time_power = [tp for tp in time_power if len(tp) == 2]
    times = [float(t)*scaling for t,p in time_power]
    powers = [float(p) for t,p in time_power][1:-1] # skip first/last values bc they're misleading
    avgpow = sum(powers)/len(powers)
    return times[-1]-times[0], avgpow # end-start


power = []

for t,td in tests_dict.items():
    power.append(parse_hier_power_rpt(td))

power = pd.DataFrame(power,   #  mW
                     columns=['Leakage','Internal','Switching','Total'],
                     index=tests_dict.keys()) # type: ignore


time_ns = [parse_power_profile(td)[0] for td in tests_dict.values()]
energy = power.mul(time_ns,axis=0) / num_inputs # mW * ns = pJ
energy.columns = [c+' Energy (pJ)' for c in energy.columns]
database = pd.concat([energy,power],axis=1)

#database.insert(0,'output_af',[td['output_af'] for td in tests_dict.values()])
database.insert(0,'time_ns',time_ns)
database.insert(0,'test',[i.split('-')[1] for i in database.index])
database.insert(0,'design',[i.split('-')[0] for i in database.index])

#print(results_for_plots)
database.to_hdf(PDK+'.h5',key='df',mode='w')
database


,design,test,time_ns,Leakage Energy (pJ),Internal Energy (pJ),Switching Energy (pJ),Total Energy (pJ),Leakage,Internal,Switching,Total
regfile8x64-10ns-reset-rand,regfile8x64,10ns,75.0,0.000434,27.112350,5.109802,32.222550,0.000006,0.361498,0.068131,0.429634
regfile8x64-10ns-reset,regfile8x64,10ns,75.0,0.000429,20.760825,3.738015,24.499275,0.000006,0.276811,0.049840,0.326657
regfile8x64-10ns-read-rand,regfile8x64,10ns,75.0,0.000434,20.698875,347.214750,367.914000,0.000006,0.275985,4.629530,4.905520
regfile8x64-10ns-read,regfile8x64,10ns,75.0,0.000429,2.933932,6.251985,9.186375,0.000006,0.039119,0.083360,0.122485
regfile8x64-10ns-write-rand,regfile8x64,10ns,75.0,0.000438,31.711875,10.371600,42.083925,0.000006,0.422825,0.138288,0.561119
regfile8x64-10ns-write,regfile8x64,10ns,75.0,0.000429,22.168650,3.952853,26.121975,0.000006,0.295582,0.052705,0.348293
regfile32x64-10ns-reset-rand,regfile32x64,10ns,315.0,0.006397,134.285760,33.032475,167.324535,0.000020,0.426304,0.104865,0.531189
regfile32x64-10ns-reset,regfile32x64,10ns,315.0,0.006305,113.242185,28.200785,141.449175,0.000020,0.359499,0.089526,0.449045
regfile32x64-10ns-read-rand,regfile32x64,10ns,315.0,0.006446,121.699620,1651.396950,1773.100350,0.000020,0.386348,5.242530,5.628890
regfile32x64-10ns-read,regfile32x64,10ns,315.0,0.006296,35.217945,43.769250,78.993180,0.000020,0.111803,0.138950,0.250772
